In [1]:
from socket import *
import hashlib
import os
import time

In [2]:
def generate_md5_hash (file_data):
    md5_hash = hashlib.md5(file_data)
    f_id = md5_hash.hexdigest()
    return str(f_id)

In [69]:
serverPort = 7701
serverURL = "localhost"
FORMAT = "UTF-8"
# 
# Create TCP socket for future connections
#
clientSocket = socket(AF_INET, SOCK_STREAM)
# clientSocket.TCPServer.allow_reuse_address = True
# 
# Connect the client to the specified server
#
clientSocket.connect((serverURL, serverPort))
print("Client connected to server: " + serverURL + ":" + str(serverPort))

Client connected to server: localhost:7701


In [56]:
clientSocket.send(b'LIST_FILES')
result = clientSocket.recv(4096)
result = result.decode("UTF-8")
print("Result from server: " + result)

Result from server: There are no files available


In [70]:
clientSocket.send(b'UPLOAD')
result = clientSocket.recv(4096)
result = result.decode("UTF-8")
print("Message from server: " +result)
if result:
    filename = 'The_file.jpg'
    filesize = os.path.getsize(filename)
    message = bytes(filename + ';'+str(filesize), FORMAT)
    print(message)
    clientSocket.send(message)
    result = clientSocket.recv(4096)
    result = result.decode(FORMAT)
    print("Message from server: "+ result)
    # import pdb; pdb.set_trace()
    if result == 'READY':
        data = open(filename, 'rb')
        data = data.read()
        clientSocket.sendall(data)
        hash_rec = clientSocket.recv(4096)
        # hash_rec = hash.decode("ascii")
        hash_gen = generate_md5_hash(data)
        print(hash_gen)
        print(hash_rec)
        if hash_rec.decode('utf-8') == hash_gen:
            print("correct")
            clientSocket.send(b'SUCCESS')

Message from server: Please send the file name and file size
b'The_file.jpg;194560'
Message from server: READY
6008994857f03e089549f58db518f5d9
b'6008994857f03e089549f58db518f5d9'
correct


In [64]:
clientSocket.send(b'LIST_FILES')
result = clientSocket.recv(1024)
result = result.decode("UTF-8")
print("Result from server: " + result)
# time.sleep(10)

Result from server: 6008994857f03e089549f58db518f5d9;The_file.jpg;194560


In [66]:
fileid_rec = '6008994857f03e089549f58db518f5d9'
clientSocket.send(b'DOWNLOAD')
result = clientSocket.recv(1024)
result = result.decode("UTF-8")
print("Message from server: " + result)
clientSocket.send(bytes(fileid_rec, 'utf-8'))

data = b''
data += clientSocket.recv(65536)
data += clientSocket.recv(65536)
data += clientSocket.recv(65536)
# import pdb;pdb.set_trace()
hash = generate_md5_hash(data)
if hash == fileid_rec:
    print("correct")
    clientSocket.send(b'SUCCESS')

Message from server: SEND fileid
correct
